# Applying data minimization to a trained regression ML model

# An example of xor_encryption

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
import struct
import numpy as np
from apt.minimization import XOREncryption

float_key = 123456789
string_key = "string_key_DPT"

xor_encryptor = XOREncryption(float_key, string_key)
data = np.array([[1.2, 'hello'], [300.0, 21.48]], dtype=object)

encrypted_data = xor_encryptor.encrypt(data)
print(encrypted_data)

decrypted_data = xor_encryptor.decrypt(encrypted_data)
print(decrypted_data)

[[123345365 '\x1b\x11\x1e\x05\x01']
 [110235285 125504437]]
[[1.2 'hello']
 [300.0 21.48]]


In this tutorial we will show how to perform data minimization for regression ML models using the minimization module.

We will show you applying data minimization to a different trained regression models.

## Load data
QI parameter determines which features will be minimized.

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

dataset = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.5, random_state=14)

features = ['age', 'sex', 'bmi', 'bp',
                's1', 's2', 's3', 's4', 's5', 's6']
QI = ['age', 'bmi', 's2', 's5', 's6']

In [3]:
float_key = 123456789
string_key = "string_key_DPT"

xor_encryptor = XOREncryption(float_key, string_key)

X_train_encrypted = xor_encryptor.encrypt(X_train)
X_test_encrypted = xor_encryptor.encrypt(X_test)
y_train_encrypted = xor_encryptor.encrypt(y_train)
y_test_encrypted = xor_encryptor.encrypt(y_test)


In [4]:
y_train

array([104., 137., 190., 220., 171.,  70., 128., 292., 178., 127., 310.,
       150.,  39.,  65., 110.,  53.,  71.,  77.,  47., 175., 275., 283.,
        77.,  97.,  92., 258.,  66., 202., 230., 220., 182., 103., 217.,
       277., 281., 142.,  63., 137.,  90., 139.,  63., 140., 332.,  71.,
       225.,  93., 268.,  99.,  88., 182., 232., 162., 293.,  90.,  71.,
        51.,  77., 124., 190., 152., 212., 115., 116., 179.,  96., 139.,
       192.,  42., 180., 111., 177.,  81., 198., 131., 230., 197.,  64.,
       321., 275., 214., 210., 122., 141., 121., 191., 126., 168., 277.,
       111.,  68., 265., 172., 129.,  84., 153., 174., 252., 196., 196.,
       185., 262., 104., 168., 113., 232., 186., 346.,  65., 261.,  51.,
        72.,  55.,  55., 156.,  84., 265., 199., 135.,  69.,  68., 180.,
       170.,  48., 170., 155., 132.,  55.,  83., 107.,  78., 281., 144.,
       235., 242.,  49., 109., 114., 198.,  80.,  75.,  95., 263., 179.,
       109., 281., 198.,  59., 174.,  81.,  72., 31

In [5]:
xor_encryptor.decrypt(y_train_encrypted)

array([104.0, 137.0, 190.0, 220.0, 171.0, 70.0, 128.0, 292.0, 178.0,
       127.0, 310.0, 150.0, 39.0, 65.0, 110.0, 53.0, 71.0, 77.0, 47.0,
       175.0, 275.0, 283.0, 77.0, 97.0, 92.0, 258.0, 66.0, 202.0, 230.0,
       220.0, 182.0, 103.0, 217.0, 277.0, 281.0, 142.0, 63.0, 137.0, 90.0,
       139.0, 63.0, 140.0, 332.0, 71.0, 225.0, 93.0, 268.0, 99.0, 88.0,
       182.0, 232.0, 162.0, 293.0, 90.0, 71.0, 51.0, 77.0, 124.0, 190.0,
       152.0, 212.0, 115.0, 116.0, 179.0, 96.0, 139.0, 192.0, 42.0, 180.0,
       111.0, 177.0, 81.0, 198.0, 131.0, 230.0, 197.0, 64.0, 321.0, 275.0,
       214.0, 210.0, 122.0, 141.0, 121.0, 191.0, 126.0, 168.0, 277.0,
       111.0, 68.0, 265.0, 172.0, 129.0, 84.0, 153.0, 174.0, 252.0, 196.0,
       196.0, 185.0, 262.0, 104.0, 168.0, 113.0, 232.0, 186.0, 346.0,
       65.0, 261.0, 51.0, 72.0, 55.0, 55.0, 156.0, 84.0, 265.0, 199.0,
       135.0, 69.0, 68.0, 180.0, 170.0, 48.0, 170.0, 155.0, 132.0, 55.0,
       83.0, 107.0, 78.0, 281.0, 144.0, 235.0, 242.0, 49.0

## Train DecisionTreeRegressor model

In [6]:
from sklearn.tree import DecisionTreeRegressor

model1 = DecisionTreeRegressor(random_state=10, min_samples_split=2)
model1.fit(X_train_encrypted, y_train_encrypted)
print('Base model accuracy (R2 score): ', model1.score(X_test_encrypted, y_test_encrypted))

Base model accuracy (R2 score):  -0.5744560891286405


## Run minimization
We will try to run minimization with only a subset of the features.

In [7]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
from apt.minimization import GeneralizeToRepresentative
# note that is_regression param is True

minimizer1 = GeneralizeToRepresentative(model1, target_accuracy=0.7, is_regression=True,
                                    features_to_minimize=QI)

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train1, x_test1, y_generalizer_train1, y_test1 = train_test_split(X_test_encrypted, y_test_encrypted,
                                                                test_size = 0.4, random_state = 38)

x_train_predictions1 = model1.predict(X_generalizer_train1)
minimizer1.fit(X_generalizer_train1, x_train_predictions1, features_names=features)
transformed1 = minimizer1.transform(x_test1, features_names=features)
print('Accuracy on minimized data: ', model1.score(transformed1, y_test1))
print('generalizations: ',minimizer1.generalizations)#%% md

/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): -0.299939
Improving accuracy


/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

...

/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

feature to remove: s5
Removed feature: s5, new relative accuracy: 0.952321


/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

Accuracy on minimized data:  -0.622764252127374
generalizations:  {'ranges': {'age': [-123461900.0, -123459620.0, -123455836.0, -123455460.0, -123455320.0, -1800.0, -1344.0, 3312.0, 123455240.0, 123456464.0, 123457308.0, 123458488.0, 123458768.0]}, 'categories': {}, 'untouched': ['s4', 's1', 'bp', 's6', 'bmi', 'sex', 's2', 's5', 's3'], 'category_representatives': {}, 'range_representatives': {'age': [-123461900.0, -123463970, -123460824, -123457157, 70.0, -123455421, -123454508, 2328.0, 61725964.0, 123454503, 123455973, 123457168, 123458291]}}


## Train linear regression model

In [8]:
from sklearn.linear_model import LinearRegression
from apt.minimization import GeneralizeToRepresentative

model2 = LinearRegression()
model2.fit(X_train_encrypted, y_train_encrypted)
print('Base model accuracy (R2 score): ', model2.score(X_test_encrypted, y_test_encrypted))

Base model accuracy (R2 score):  0.07349787918321149


## Run minimization
We will try to run minimization with only a subset of the features.

In [9]:
# note that is_regression param is True

minimizer2 = GeneralizeToRepresentative(model2, target_accuracy=0.7, is_regression=True,
                                    features_to_minimize=QI)

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train2, x_test2, y_generalizer_train2, y_test2 = train_test_split(X_test_encrypted, y_test_encrypted,
                                                                test_size = 0.4, random_state = 38)

x_train_predictions2 = model2.predict(X_generalizer_train2)
minimizer2.fit(X_generalizer_train2, x_train_predictions2, features_names=features)
transformed2 = minimizer2.transform(x_test2, features_names=features)

/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.979566
Improving generalizations


/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

Pruned tree to level: 1, new relative accuracy: 0.979566
Pruned tree to level: 2, new relative accuracy: 0.979565


/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

...

/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

Pruned tree to level: 6, new relative accuracy: 0.974075
Pruned tree to level: 7, new relative accuracy: 0.971941
Pruned tree to level: 8, new relative accuracy: 0.975746


/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

Pruned tree to level: 9, new relative accuracy: 0.946310


/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
/Users/qidingran/anaconda3/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X d

In [10]:
print('Accuracy on minimized data: ', model2.score(transformed2, y_test2))
print('generalizations: ',minimizer2.generalizations)

Accuracy on minimized data:  0.11718580351366281
generalizations:  {'ranges': {'age': [], 'bmi': [-123461408.0, -20.0, -12.0], 's2': [-123460028.0, -123454288.0, -356.0], 's5': [12.0], 's6': []}, 'categories': {}, 'untouched': ['s4', 's1', 'bp', 'sex', 's3'], 'category_representatives': {}, 'range_representatives': {'age': [], 'bmi': [-123461408.0, -123461503, -123454790], 's2': [-123460028.0, -123461452, -123456111], 's5': [12.0], 's6': []}}
